## Table Layout with PyFixest

Pyfixest comes with functions to generate publication-ready tables. Regression tables are generated with `pf.etable()`, which can output different formats, for instance using the [Great Tables](https://posit-dev.github.io/great-tables/articles/intro.html) package or generating formatted LaTex Tables using [booktabs](https://ctan.org/pkg/booktabs?lang=en). There are also further functions `pf.dtable()` to display descriptive statistics and `pf.make_table()` generating formatted tables from pandas dataframes in the same layout. 

Contents: 

-   [Regression Tables via `pf.etable()`](#regression-tables-via-pfetable)

    -   [Basic usage](#basic-usage)
    -   [Keep and drop variables](#keep-and-drop-variables)
    -   [Hide fixed effects or SE-type rows](#hide-fixed-effects-or-se-type-rows)
    -   [Display p-values or confidence intervals](#display-p-values-or-confidence-intervals)
    -   [Significance levels and rounding](#significance-levels-and-rounding)
    -   [Other output formats](#other-output-formats)
    -   [Rename variables](#rename-variables)
    -   [Custom model headlines](#custom-model-headlines)
    -   [Further custom model information](#further-custom-model-information)
    -   [Custom table notes](#custom-table-notes)
    -   [Publication-ready LaTex tables](#publication-ready-latex-tables)
    -   [Rendering Tables in Quarto](#rendering-tables-in-quarto)
    
-   [Descriptive Statistics via `pf.dtable()`](#descriptive-statistics-via-pfdtable)
    -   [Basic usage of dtable](#basic-usage-of-dtable)
    -   [Summarize by groups in columns and rows](#summarize-by-groups-in-columns-and-rows)

-   [Table Layout for DataFrames with `pf.make_table()`](#table-layout-for-dataframes-with-pfmake_table)
    -  [Basic usage of make_table](#basic-usage-of-make_table)
    -  [Multiindex DataFrames](#multiindex-dataframes)

-   [Custom Styling with Great Tables](#custom-styling-with-great-tables)
    -   [Example Styling](#example-styling)
    -   [Defining Table Styles](#defining-table-styles)


To begin, we load some libraries and fit a set of regression models. 

In [1]:
import numpy as np
import pandas as pd
import pylatex as pl  # for the latex table; note: not a dependency of pyfixest - needs manual installation
from great_tables import loc, style
from IPython.display import FileLink, display

import pyfixest as pf

%load_ext autoreload
%autoreload 2

data = pf.get_data()

fit1 = pf.feols("Y ~ X1 + X2 | f1", data=data)
fit2 = pf.feols("Y ~ X1 + X2 | f1 + f2", data=data)
fit3 = pf.feols("Y ~ X1 *X2 | f1 + f2", data=data)
fit4 = pf.feols("Y2 ~ X1 + X2 | f1", data=data)
fit5 = pf.feols("Y2 ~ X1 + X2 | f1 + f2", data=data)
fit6 = pf.feols("Y2 ~ X1 *X2 | f1 + f2", data=data)

# Regression Tables via `pf.etable()`

## Basic Usage

We can compare all regression models via the pyfixest-internal `pf.etable()` function: 

In [2]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6])

GT(_tbl_data=  level_0        level_1                       0                       1  \
0    coef             X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1    coef             X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2    coef          X1:X2                                                   
3      fe             f1                       x                       x   
4      fe             f2                       -                       x   
5   stats   Observations                     997                     997   
6   stats      S.E. type                  by: f1                  by: f1   
7   stats  R<sup>2</sup>                   0.489                   0.659   

                        2                       3                       4  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2      0.011 <br> (0.018)                                                   
3                       x                       x                       x   
4                       x                       -                       x   
5                     997                     998                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.120                   0.172   

                        5  
0  -1.231*** <br> (0.192)  
1     -0.074 <br> (0.104)  
2     -0.041 <br> (0.081)  
3                       x  
4                       x  
5                     998  
6                  by: f1  
7                   0.172  , _body=<great_tables._gt_data.Body object at 0x000001BC689B2510>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC688530D0>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC68836950>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font

You can also estimate and display multiple regressions with one line of code using the (py)fixest stepwise notation:

In [3]:
pf.etable(pf.feols("Y+Y2~csw(X1,X2,X1:X2)", data=data))

GT(_tbl_data=  level_0        level_1                       0                       1  \
0    coef             X1  -1.000*** <br> (0.085)  -0.993*** <br> (0.082)   
1    coef             X2                          -0.176*** <br> (0.022)   
2    coef          X1:X2                                                   
3    coef      Intercept   0.919*** <br> (0.112)   0.889*** <br> (0.108)   
4   stats   Observations                     998                     998   
5   stats      S.E. type                     iid                     iid   
6   stats  R<sup>2</sup>                   0.123                   0.177   

                        2                       3                       4  \
0  -0.992*** <br> (0.082)  -1.322*** <br> (0.215)  -1.316*** <br> (0.214)   
1  -0.197*** <br> (0.036)                            -0.133* <br> (0.057)   
2      0.020 <br> (0.027)                                                   
3   0.888*** <br> (0.108)   1.064*** <br> (0.283)   1.042*** <br> (0.283)   
4                     998                     999                     999   
5                     iid                     iid                     iid   
6                   0.177                   0.037                   0.042   

                        5  
0  -1.316*** <br> (0.215)  
1     -0.132 <br> (0.095)  
2     -0.001 <br> (0.071)  
3   1.042*** <br> (0.283)  
4                     999  
5                     iid  
6                   0.042  , _body=<great_tables._gt_data.Body object at 0x000001BC68CFF2D0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC68846310>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC68845350>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue'

## Keep and drop variables
`etable` allows us to do a few things out of the box. For example, we can only keep the variables that we'd like, which keeps all variables that fit the provided regex match. 

In [4]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], keep="X1")

GT(_tbl_data=  level_0        level_1                       0                       1  \
0    coef             X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1    coef          X1:X2                                                   
2      fe             f1                       x                       x   
3      fe             f2                       -                       x   
4   stats   Observations                     997                     997   
5   stats      S.E. type                  by: f1                  by: f1   
6   stats  R<sup>2</sup>                   0.489                   0.659   

                        2                       3                       4  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1      0.011 <br> (0.018)                                                   
2                       x                       x                       x   
3                       x                       -                       x   
4                     997                     998                     998   
5                  by: f1                  by: f1                  by: f1   
6                   0.659                   0.120                   0.172   

                        5  
0  -1.231*** <br> (0.192)  
1     -0.041 <br> (0.081)  
2                       x  
3                       x  
4                     998  
5                  by: f1  
6                   0.172  , _body=<great_tables._gt_data.Body object at 0x000001BC68D4F610>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC6883A7D0>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC68D4E950>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue'

We can use the `exact_match` argument to select a specific set of variables: 

In [5]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], keep=["X1", "X2"], exact_match=True)

GT(_tbl_data=  level_0        level_1                       0                       1  \
0    coef             X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1    coef             X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2      fe             f1                       x                       x   
3      fe             f2                       -                       x   
4   stats   Observations                     997                     997   
5   stats      S.E. type                  by: f1                  by: f1   
6   stats  R<sup>2</sup>                   0.489                   0.659   

                        2                       3                       4  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2                       x                       x                       x   
3                       x                       -                       x   
4                     997                     998                     998   
5                  by: f1                  by: f1                  by: f1   
6                   0.659                   0.120                   0.172   

                        5  
0  -1.231*** <br> (0.192)  
1     -0.074 <br> (0.104)  
2                       x  
3                       x  
4                     998  
5                  by: f1  
6                   0.172  , _body=<great_tables._gt_data.Body object at 0x000001BC68D6ECD0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC68D47150>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC68CAC890>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue'

We can also easily **drop** variables via the `drop` argument: 

In [6]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], drop=["X1"])

GT(_tbl_data=  level_0        level_1                       0                       1  \
0    coef             X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
1      fe             f1                       x                       x   
2      fe             f2                       -                       x   
3   stats   Observations                     997                     997   
4   stats      S.E. type                  by: f1                  by: f1   
5   stats  R<sup>2</sup>                   0.489                   0.659   

                        2                      3                      4  \
0  -0.185*** <br> (0.025)  -0.131** <br> (0.042)  -0.118** <br> (0.042)   
1                       x                      x                      x   
2                       x                      -                      x   
3                     997                    998                    998   
4                  by: f1                 by: f1                 by: f1   
5                   0.659                  0.120                  0.172   

                     5  
0  -0.074 <br> (0.104)  
1                    x  
2                    x  
3                  998  
4               by: f1  
5                0.172  , _body=<great_tables._gt_data.Body object at 0x000001BC68863150>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC68D71810>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC68D72910>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value=

## Hide fixed effects or SE-type rows
We can hide the rows showing the relevant fixed effects and those showing the S.E. type by setting `show_fe=False` and `show_setype=False` (for instance when the set of fixed effects or the estimation method for the std. errors is the same for all models and you want to describe this in the text or table notes rather than displaying it in the table). 

In [7]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], show_fe=False, show_se_type=False)

GT(_tbl_data=  level_0        level_1                       0                       1  \
0    coef             X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)   
1    coef             X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)   
2    coef          X1:X2                                                   
3   stats   Observations                     997                     997   
4   stats  R<sup>2</sup>                   0.489                   0.659   

                        2                       3                       4  \
0  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)   
1  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)   
2      0.011 <br> (0.018)                                                   
3                     997                     998                     998   
4                   0.659                   0.120                   0.172   

                        5  
0  -1.231*** <br> (0.192)  
1     -0.074 <br> (0.104)  
2     -0.041 <br> (0.081)  
3                     998  
4                   0.172  , _body=<great_tables._gt_data.Body object at 0x000001BC68CAED10>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC6886BB90>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC6886F4D0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, catego

## Display p-values or confidence intervals
By default, `pf.etable()` reports **standard errors**. But we can also ask to output p-values or confidence intervals via the `coef_fmt` function argument. 

In [8]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], coef_fmt="b \n (se) \n [p]")

GT(_tbl_data=  level_0        level_1                                    0  \
0    coef             X1  -0.950*** <br> (0.067) <br> [0.000]   
1    coef             X2  -0.174*** <br> (0.018) <br> [0.000]   
2    coef          X1:X2                                        
3      fe             f1                                    x   
4      fe             f2                                    -   
5   stats   Observations                                  997   
6   stats      S.E. type                               by: f1   
7   stats  R<sup>2</sup>                                0.489   

                                     1                                    2  \
0  -0.924*** <br> (0.061) <br> [0.000]  -0.924*** <br> (0.061) <br> [0.000]   
1  -0.174*** <br> (0.015) <br> [0.000]  -0.185*** <br> (0.025) <br> [0.000]   
2                                           0.011 <br> (0.018) <br> [0.565]   
3                                    x                                    x   
4                                    x                                    x   
5                                  997                                  997   
6                               by: f1                               by: f1   
7                                0.659                                0.659   

                                     3                                    4  \
0  -1.267*** <br> (0.174) <br> [0.000]  -1.232*** <br> (0.192) <br> [0.000]   
1   -0.131** <br> (0.042) <br> [0.005]   -0.118** <br> (0.042) <br> [0.008]   
2                                                                             
3                                    x                                    x   
4                                    -                                    x   
5                                  998                                  998   
6                               by: f1                               by: f1   
7                                0.120                                0.172   

                                     5  
0  -1.231*** <br> (0.192) <br> [0.000]  
1     -0.074 <br> (0.104) <br> [0.482]  
2     -0.041 <br> (0.081) <br> [0.618]  
3                                    x  
4                                    x  
5                                  998  
6                               by: f1  
7                                0.172  , _body=<great_tables._gt_data.Body object at 0x000001BC66EF6CD0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC688549D0>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error) \

## Significance levels and rounding
Additionally, we can also overwrite the defaults for the reported significance levels and control the rounding of results via the `signif_code` and `digits` function arguments: 

In [9]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], signif_code=[0.01, 0.05, 0.1], digits=5)

GT(_tbl_data=  level_0        level_1                           0  \
0    coef             X1  -0.94953*** <br> (0.06652)   
1    coef             X2  -0.17423*** <br> (0.01840)   
2    coef          X1:X2                               
3      fe             f1                           x   
4      fe             f2                           -   
5   stats   Observations                         997   
6   stats      S.E. type                      by: f1   
7   stats  R<sup>2</sup>                     0.48899   

                            1                           2  \
0  -0.92405*** <br> (0.06093)  -0.92417*** <br> (0.06094)   
1  -0.17411*** <br> (0.01461)  -0.18550*** <br> (0.02516)   
2                                  0.01057 <br> (0.01818)   
3                           x                           x   
4                           x                           x   
5                         997                         997   
6                      by: f1                      by: f1   
7                     0.65904                     0.65916   

                            3                           4  \
0  -1.26655*** <br> (0.17359)  -1.23153*** <br> (0.19228)   
1  -0.13056*** <br> (0.04239)  -0.11767*** <br> (0.04152)   
2                                                           
3                           x                           x   
4                           -                           x   
5                         998                         998   
6                      by: f1                      by: f1   
7                     0.12017                     0.17151   

                            5  
0  -1.23100*** <br> (0.19167)  
1     -0.07369 <br> (0.10356)  
2     -0.04082 <br> (0.08093)  
3                           x  
4                           x  
5                         998  
6                      by: f1  
7                     0.17180  , _body=<great_tables._gt_data.Body object at 0x000001BC68D46D10>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC6884D250>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.1, ** p < 0.05, *** p < 0.01. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC6878C110>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left

## Other output formats
By default, `pf.etable()` returns a GT object (see [the Great Tables package](https://posit-dev.github.io/great-tables/articles/intro.html)), but you can also opt to dataframe, markdown, or latex output via the `type` argument.

In [10]:
# Pandas styler output:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    signif_code=[0.01, 0.05, 0.1],
    digits=5,
    coef_fmt="b (se)",
    type="df",
)

,est1,est2,est3,est4,est5,est6
depvar,Y,Y,Y,Y2,Y2,Y2
X1,-0.94953*** (0.06652),-0.92405*** (0.06093),-0.92417*** (0.06094),-1.26655*** (0.17359),-1.23153*** (0.19228),-1.23100*** (0.19167)
X2,-0.17423*** (0.01840),-0.17411*** (0.01461),-0.18550*** (0.02516),-0.13056*** (0.04239),-0.11767*** (0.04152),-0.07369 (0.10356)
X1:X2,,,0.01057 (0.01818),,,-0.04082 (0.08093)
f1,x,x,x,x,x,x
f2,-,x,x,-,x,x
Observations,997,997,997,998,998,998
S.E. type,by: f1,by: f1,by: f1,by: f1,by: f1,by: f1
R2,0.48899,0.65904,0.65916,0.12017,0.17151,0.17180


In [11]:
# Markdown output:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    signif_code=[0.01, 0.05, 0.1],
    digits=5,
    type="md",
)

index                 est1          est2          est3          est4          est5          est6
------------  ------------  ------------  ------------  ------------  ------------  ------------
depvar                   Y             Y             Y            Y2            Y2            Y2
------------------------------------------------------------------------------------------------
X1            -0.94953***   -0.92405***   -0.92417***   -1.26655***   -1.23153***   -1.23100***
                 (0.06652)     (0.06093)     (0.06094)     (0.17359)     (0.19228)     (0.19167)
X2            -0.17423***   -0.17411***   -0.18550***   -0.13056***   -0.11767***      -0.07369
                 (0.01840)     (0.01461)     (0.02516)     (0.04239)     (0.04152)     (0.10356)
X1:X2                                         0.01057                                  -0.04082
                                             (0.01818)                                 (0.08093)
---------------------------------

To obtain latex output use `format = "tex"`. If you want to save the table as a tex file, you can use the `filename=` argument to specify the respective path where it should be saved. If you want the latex code to be displayed in the notebook, you can use the `print_tex=True` argument.
Etable will use latex packages `booktabs`, `threeparttable` and `makecell` for the table layout, so don't forget to include these packages in your latex document. 
 

In [12]:
# LaTex output (include latex packages booktabs, threeparttable, and makecell in your document):
tab = pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    signif_code=[0.01, 0.05, 0.1],
    digits=2,
    type="tex",
    print_tex=True,
)

The following code generates a pdf including the regression table which you can display clicking on the link below the cell:

In [13]:
## Use pylatex to create a tex file with the table


def make_pdf(tab, file):
    "Create a PDF document with tex table."
    doc = pl.Document()
    doc.packages.append(pl.Package("booktabs"))
    doc.packages.append(pl.Package("threeparttable"))
    doc.packages.append(pl.Package("makecell"))

    with (
        doc.create(pl.Section("A PyFixest LateX Table")),
        doc.create(pl.Table(position="htbp")) as table,
    ):
        table.append(pl.NoEscape(tab))

    doc.generate_pdf(file, clean_tex=False)


# Compile latex to pdf & display a button with the hyperlink to the pdf
make_pdf(tab, "latexdocs/SampleTableDoc")
display(FileLink("latexdocs/SampleTableDoc.pdf"))

c:\PythonProjects\pyfixest\docs\latexdocs\SampleTableDoc.pdf

## Rename variables
You can also rename variables if you want to have a more readable output. Just pass a dictionary to the `labels` argument. Note that interaction terms will also be relabeled using the specified labels for the interacted variables (if you want to manually relabel an interaction term differently, add it to the dictionary).

In [14]:
labels = {
    "Y": "Wage",
    "Y2": "Wealth",
    "X1": "Age",
    "X2": "Years of Schooling",
    "f1": "Industry",
    "f2": "Year",
}

pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], labels=labels)

GT(_tbl_data=  level_0                        level_1                       0  \
0    coef                            Age  -0.950*** <br> (0.067)   
1    coef             Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age &#215; Years of Schooling                           
3      fe                       Industry                       x   
4      fe                           Year                       -   
5   stats                   Observations                     997   
6   stats                      S.E. type                  by: f1   
7   stats                  R<sup>2</sup>                   0.489   

                        1                       2                       3  \
0  -0.924*** <br> (0.061)  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)   
1  -0.174*** <br> (0.015)  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   
2                              0.011 <br> (0.018)                           
3                       x                       x                       x   
4                       x                       x                       -   
5                     997                     997                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.659                   0.120   

                        4                       5  
0  -1.232*** <br> (0.192)  -1.231*** <br> (0.192)  
1   -0.118** <br> (0.042)     -0.074 <br> (0.104)  
2                             -0.041 <br> (0.081)  
3                       x                       x  
4                       x                       x  
5                     998                     998  
6                  by: f1                  by: f1  
7                   0.172                   0.172  , _body=<great_tables._gt_data.Body object at 0x000001BC69030850>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC68F2F190>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC6A0F0D10>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scs

If you want to label the rows indicating the inclusion of fixed effects not with the variable label but with a custom label, you can pass on a separate dictionary to the `felabels` argument.

In [15]:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    labels=labels,
    felabels={"f1": "Industry Fixed Effects", "f2": "Year Fixed Effects"},
)

GT(_tbl_data=  level_0                        level_1                       0  \
0    coef                            Age  -0.950*** <br> (0.067)   
1    coef             Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age &#215; Years of Schooling                           
3      fe         Industry Fixed Effects                       x   
4      fe             Year Fixed Effects                       -   
5   stats                   Observations                     997   
6   stats                      S.E. type                  by: f1   
7   stats                  R<sup>2</sup>                   0.489   

                        1                       2                       3  \
0  -0.924*** <br> (0.061)  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)   
1  -0.174*** <br> (0.015)  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   
2                              0.011 <br> (0.018)                           
3                       x                       x                       x   
4                       x                       x                       -   
5                     997                     997                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.659                   0.120   

                        4                       5  
0  -1.232*** <br> (0.192)  -1.231*** <br> (0.192)  
1   -0.118** <br> (0.042)     -0.074 <br> (0.104)  
2                             -0.041 <br> (0.081)  
3                       x                       x  
4                       x                       x  
5                     998                     998  
6                  by: f1                  by: f1  
7                   0.172                   0.172  , _body=<great_tables._gt_data.Body object at 0x000001BC68892B90>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC686B3C50>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC686B1B50>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scs

## Custom model headlines
You can also add custom headers for each model by passing a list of strings to the `model_headers` argument.

In [16]:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    labels=labels,
    model_heads=["US", "China", "EU", "US", "China", "EU"],
)

GT(_tbl_data=  level_0                        level_1                       0  \
0    coef                            Age  -0.950*** <br> (0.067)   
1    coef             Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age &#215; Years of Schooling                           
3      fe                       Industry                       x   
4      fe                           Year                       -   
5   stats                   Observations                     997   
6   stats                      S.E. type                  by: f1   
7   stats                  R<sup>2</sup>                   0.489   

                        1                       2                       3  \
0  -0.924*** <br> (0.061)  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)   
1  -0.174*** <br> (0.015)  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   
2                              0.011 <br> (0.018)                           
3                       x                       x                       x   
4                       x                       x                       -   
5                     997                     997                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.659                   0.659                   0.120   

                        4                       5  
0  -1.232*** <br> (0.192)  -1.231*** <br> (0.192)  
1   -0.118** <br> (0.042)     -0.074 <br> (0.104)  
2                             -0.041 <br> (0.081)  
3                       x                       x  
4                       x                       x  
5                     998                     998  
6                  by: f1                  by: f1  
7                   0.172                   0.172  , _body=<great_tables._gt_data.Body object at 0x000001BC68248410>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC683BF190>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=2, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=2, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None), SpannerInfo(spanner_id='US', spanner_level=1, spanner_label='US', spanner_units=None, spanner_pattern=None, vars=['0', '3'], built=None), SpannerInfo(spanner_id='China', spanner_level=1, spanner_label='China', spanner_units=None, spanner_pattern=None, vars=['1', '4'], built=None), SpannerInfo(spanner_id='EU', spanner_level=1, spanner_label='EU', spanner_units=None, spanner_pattern=None, vars=['2', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC68346690>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(sc

Or change the ordering of headlines having headlines first and then dependent variables using the `head_order` argument. "hd" stands for headlines then dependent variables, "dh" for dependent variables then headlines. Assigning "d" or "h" can be used to only show dependent variables or only headlines. When head_order="" only model numbers are shown.

In [17]:
pf.etable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="hd",
)

GT(_tbl_data=  level_0                        level_1                       0  \
0    coef                            Age  -0.950*** <br> (0.067)   
1    coef             Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age &#215; Years of Schooling                           
3      fe                       Industry                       x   
4      fe                           Year                       -   
5   stats                   Observations                     997   
6   stats                      S.E. type                  by: f1   
7   stats                  R<sup>2</sup>                   0.489   

                        1                       2                       3  \
0  -1.267*** <br> (0.174)  -0.924*** <br> (0.061)  -1.232*** <br> (0.192)   
1   -0.131** <br> (0.042)  -0.174*** <br> (0.015)   -0.118** <br> (0.042)   
2                                                                           
3                       x                       x                       x   
4                       -                       x                       x   
5                     998                     997                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.120                   0.659                   0.172   

                        4                       5  
0  -0.924*** <br> (0.061)  -1.231*** <br> (0.192)  
1  -0.185*** <br> (0.025)     -0.074 <br> (0.104)  
2      0.011 <br> (0.018)     -0.041 <br> (0.081)  
3                       x                       x  
4                       x                       x  
5                     997                     998  
6                  by: f1                  by: f1  
7                   0.659                   0.172  , _body=<great_tables._gt_data.Body object at 0x000001BC683E2510>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC684DE190>, _spanners=Spanners([SpannerInfo(spanner_id='US', spanner_level=2, spanner_label='US', spanner_units=None, spanner_pattern=None, vars=['0', '1'], built=None), SpannerInfo(spanner_id='China', spanner_level=2, spanner_label='China', spanner_units=None, spanner_pattern=None, vars=['2', '3'], built=None), SpannerInfo(spanner_id='EU', spanner_level=2, spanner_label='EU', spanner_units=None, spanner_pattern=None, vars=['4', '5'], built=None), SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '2', '4'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['1', '3', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC684DCA90>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(sc

Remove the dependent variables from the headers:

In [18]:
pf.etable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="",
)

GT(_tbl_data=  level_0                        level_1                     (1)  \
0    coef                            Age  -0.950*** <br> (0.067)   
1    coef             Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age &#215; Years of Schooling                           
3      fe                       Industry                       x   
4      fe                           Year                       -   
5   stats                   Observations                     997   
6   stats                      S.E. type                  by: f1   
7   stats                  R<sup>2</sup>                   0.489   

                      (2)                     (3)                     (4)  \
0  -1.267*** <br> (0.174)  -0.924*** <br> (0.061)  -1.232*** <br> (0.192)   
1   -0.131** <br> (0.042)  -0.174*** <br> (0.015)   -0.118** <br> (0.042)   
2                                                                           
3                       x                       x                       x   
4                       -                       x                       x   
5                     998                     997                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.120                   0.659                   0.172   

                      (5)                     (6)  
0  -0.924*** <br> (0.061)  -1.231*** <br> (0.192)  
1  -0.185*** <br> (0.025)     -0.074 <br> (0.104)  
2      0.011 <br> (0.018)     -0.041 <br> (0.081)  
3                       x                       x  
4                       x                       x  
5                     997                     998  
6                  by: f1                  by: f1  
7                   0.659                   0.172  , _body=<great_tables._gt_data.Body object at 0x000001BC684DD210>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='(4)', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='(5)', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='(6)', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC66FD85D0>, _spanners=Spanners([]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC66FDAA50>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Rob

## Further custom model information
You can add further custom model statistics/information to the bottom of the table by using the `custom_stats` argument to which you pass a dictionary with the name of the row and lists of values. The length of the lists must be equal to the number of models.

In [19]:
pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    labels=labels,
    custom_model_stats={
        "Number of Clusters": [42, 42, 42, 37, 37, 37],
        "Additional Info": ["A", "A", "B", "B", "C", "C"],
    },
)

GT(_tbl_data=  level_0                        level_1                       0  \
0    coef                            Age  -0.950*** <br> (0.067)   
1    coef             Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age &#215; Years of Schooling                           
3      fe                       Industry                       x   
4      fe                           Year                       -   
5   stats             Number of Clusters                      42   
6   stats                Additional Info                       A   
7   stats                   Observations                     997   
8   stats                      S.E. type                  by: f1   
9   stats                  R<sup>2</sup>                   0.489   

                        1                       2                       3  \
0  -0.924*** <br> (0.061)  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)   
1  -0.174*** <br> (0.015)  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   
2                              0.011 <br> (0.018)                           
3                       x                       x                       x   
4                       x                       x                       -   
5                      42                      42                      37   
6                       A                       B                       B   
7                     997                     997                     998   
8                  by: f1                  by: f1                  by: f1   
9                   0.659                   0.659                   0.120   

                        4                       5  
0  -1.232*** <br> (0.192)  -1.231*** <br> (0.192)  
1   -0.118** <br> (0.042)     -0.074 <br> (0.104)  
2                             -0.041 <br> (0.081)  
3                       x                       x  
4                       x                       x  
5                      37                      37  
6                       C                       C  
7                     998                     998  
8                  by: f1                  by: f1  
9                   0.172                   0.172  , _body=<great_tables._gt_data.Body object at 0x000001BC66FA6CD0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC68161ED0>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC681624D0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table'

## Custom table notes
You can replace the default table notes with your own notes using the `notes` argument. 


In [20]:
mynotes = "Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet."
pf.etable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="hd",
    notes=mynotes,
)

GT(_tbl_data=  level_0                        level_1                       0  \
0    coef                            Age  -0.950*** <br> (0.067)   
1    coef             Years of Schooling  -0.174*** <br> (0.018)   
2    coef  Age &#215; Years of Schooling                           
3      fe                       Industry                       x   
4      fe                           Year                       -   
5   stats                   Observations                     997   
6   stats                      S.E. type                  by: f1   
7   stats                  R<sup>2</sup>                   0.489   

                        1                       2                       3  \
0  -1.267*** <br> (0.174)  -0.924*** <br> (0.061)  -1.232*** <br> (0.192)   
1   -0.131** <br> (0.042)  -0.174*** <br> (0.015)   -0.118** <br> (0.042)   
2                                                                           
3                       x                       x                       x   
4                       -                       x                       x   
5                     998                     997                     998   
6                  by: f1                  by: f1                  by: f1   
7                   0.120                   0.659                   0.172   

                        4                       5  
0  -0.924*** <br> (0.061)  -1.231*** <br> (0.192)  
1  -0.185*** <br> (0.025)     -0.074 <br> (0.104)  
2      0.011 <br> (0.018)     -0.041 <br> (0.081)  
3                       x                       x  
4                       x                       x  
5                     997                     998  
6                  by: f1                  by: f1  
7                   0.659                   0.172  , _body=<great_tables._gt_data.Body object at 0x000001BC68344490>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC681759D0>, _spanners=Spanners([SpannerInfo(spanner_id='US', spanner_level=2, spanner_label='US', spanner_units=None, spanner_pattern=None, vars=['0', '1'], built=None), SpannerInfo(spanner_id='China', spanner_level=2, spanner_label='China', spanner_units=None, spanner_pattern=None, vars=['2', '3'], built=None), SpannerInfo(spanner_id='EU', spanner_level=2, spanner_label='EU', spanner_units=None, spanner_pattern=None, vars=['4', '5'], built=None), SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '2', '4'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['1', '3', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit

## Publication-ready LaTex tables 
With few lines of code you thus obtain a publication-ready latex table:

In [21]:
tab = pf.etable(
    [fit1, fit4, fit2, fit5, fit3, fit6],
    labels=labels,
    model_heads=["US", "US", "China", "China", "EU", "EU"],
    head_order="hd",
    type="tex",
    notes=mynotes,
    show_fe=True,
    show_se_type=False,
    custom_model_stats={
        "Number of Clusters": [42, 42, 42, 37, 37, 37],
    },
)

# Compile latex to pdf & display a button with the hyperlink to the pdf
make_pdf(tab, "latexdocs/SampleTableDoc2")
display(FileLink("latexdocs/SampleTableDoc2.pdf"))

c:\PythonProjects\pyfixest\docs\latexdocs\SampleTableDoc2.pdf

## Rendering Tables in Quarto
When you use quarto you can include latex tables generated by pyfixest when rendering the qmd file as pdf. Just specify `output: asis` in the code block options of the respective chunk and print the LaTex string returned by etable. Don't forget to include the `\usepackage` commands for necessary latex packages in the YAML block.

Here you find a sample [qmd file](https://github.com/py-econometrics/pyfixest/blob/master/docs/quarto_example/QuartoExample.qmd) and the rendered [PDF](quarto_example/QuartoExample.pdf).

# Descriptive Statistics via `pf.dtable()`

The function `pf.dtable()` allows to display descriptive statistics for a set of variables in the same layout.

## Basic Usage of dtable
Specify the variables you want to display the descriptive statistics for. You can also use a dictionary to rename the variables and add a caption.


In [22]:
pf.dtable(
    data, vars=["Y", "Y2", "X1", "X2"], labels=labels, caption="Descriptive statistics"
)

GT(_tbl_data=                index     N   Mean Std. Dev.
0                Wage   999  -0.13      2.30
1              Wealth  1000  -0.31      5.58
2                 Age   999   1.04      0.81
3  Years of Schooling  1000  -0.13      3.05, _body=<great_tables._gt_data.Body object at 0x000001BC68320510>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='center', column_width=None), ColInfo(var='N', type=<ColInfoTypeEnum.default: 1>, column_label='N', column_align='center', column_width=None), ColInfo(var='Mean', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='center', column_width=None), ColInfo(var='Std. Dev.', type=<ColInfoTypeEnum.default: 1>, column_label='Std. Dev.', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC68110ED0>, _spanners=Spanners([]), _heading=Heading(title='Descriptive statistics', subtitle=None, preheader=None), _stubhead=None, _source_notes=[None], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC68174890>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), heading_title_font_size=OptionsInfo(scss=True, c

Choose the set of statistics to be displayed with `stats`. You can use any pandas aggregation functions. 

In [23]:
pf.dtable(
    data,
    vars=["Y", "Y2", "X1", "X2"],
    stats=["count", "mean", "std", "min", "max"],
    labels=labels,
    caption="Descriptive statistics",
)

GT(_tbl_data=                index     N   Mean Std. Dev.     Min    Max
0                Wage   999  -0.13      2.30   -6.54   6.91
1              Wealth  1000  -0.31      5.58  -16.97  17.16
2                 Age   999   1.04      0.81    0.00   2.00
3  Years of Schooling  1000  -0.13      3.05   -9.67  10.99, _body=<great_tables._gt_data.Body object at 0x000001BC66E7ECD0>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='center', column_width=None), ColInfo(var='N', type=<ColInfoTypeEnum.default: 1>, column_label='N', column_align='center', column_width=None), ColInfo(var='Mean', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='center', column_width=None), ColInfo(var='Std. Dev.', type=<ColInfoTypeEnum.default: 1>, column_label='Std. Dev.', column_align='center', column_width=None), ColInfo(var='Min', type=<ColInfoTypeEnum.default: 1>, column_label='Min', column_align='center', column_width=None), ColInfo(var='Max', type=<ColInfoTypeEnum.default: 1>, column_label='Max', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC66FA53D0>, _spanners=Spanners([]), _heading=Heading(title='Descriptive statistics', subtitle=None, preheader=None), _stubhead=None, _source_notes=[None], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC66FA7F90>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_c

## Summarize by groups in columns and rows

You can summarize by groups using the `byrow` argument when groups are to be displayed in columns. When the number of observations is the same for all variables in a group, you can also opt to display the number of observations only once for each group byin a separate line at the bottom of the table with `counts_row_below==True`.

In [24]:
# Generate some categorial data
data["country"] = np.random.choice(["US", "EU"], data.shape[0])
data["occupation"] = np.random.choice(["Blue collar", "White collar"], data.shape[0])

# Drop nan values to have balanced data
data.dropna(inplace=True)

pf.dtable(
    data,
    vars=["Y", "Y2", "X1", "X2"],
    labels=labels,
    bycol=["country", "occupation"],
    stats=["count", "mean", "std"],
    caption="Descriptive statistics",
    stats_labels={"count": "Number of observations"},
    counts_row_below=True,
)

GT(_tbl_data=  level_0                 level_1      0     1      2     3      4     5      6     7
0   stats                    Wage  -0.13  2.31  -0.33  2.39  -0.18  2.34   0.11  2.18
1   stats                  Wealth  -0.62  5.61  -0.15  5.48  -0.31  5.84  -0.20  5.41
2   stats                     Age   1.08  0.81   1.04  0.81   1.07  0.80   0.98  0.81
3   stats      Years of Schooling   0.18  2.91  -0.45  2.99  -0.25  3.14  -0.00  3.10
4    nobs  Number of observations    238          225          275          259      , _body=<great_tables._gt_data.Body object at 0x000001BC6865F750>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='Std. Dev.', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='Std. Dev.', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='Std. Dev.', column_align='center', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='center', column_width=None), ColInfo(var='7', type=<ColInfoTypeEnum.default: 1>, column_label='Std. Dev.', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC68582F10>, _spanners=Spanners([SpannerInfo(spanner_id='EU', spanner_level=2, spanner_label='EU', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2', '3'], built=None), SpannerInfo(spanner_id='US', spanner_level=2, spanner_label='US', spanner_units=None, spanner_pattern=None, vars=['4', '5', '6', '7'], built=None), SpannerInfo(spanner_id='Blue collar', spanner_level=1, spanner_label='Blue collar', spanner_units=None, spanner_pattern=None, vars=['0', '1', '4', '5'], built=None), SpannerInfo(spanner_id='White collar', spanner_level=1, spanner_label='White collar', spanner_units=None, spanner_pattern=None, vars=['2', '3', '6', '7'], built=None)]), _heading=Heading(title='Descriptive statistics', subtitle=None, preheader=None), _stubhead=None, _source_notes=[None], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC685837D0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_li

Or shows groups in columns and rows. Note that you can only use one grouping variable in rows, but several in columns (as shown above).

In [25]:
pf.dtable(
    data,
    vars=["Y", "Y2", "X1", "X2"],
    labels=labels,
    bycol=["country"],
    byrow="occupation",
    stats=["count", "mean", "std"],
    caption="Descriptive statistics",
)

GT(_tbl_data=        level_0             level_1    0      1     2    3      4     5
0   Blue collar                Wage  238  -0.13  2.31  275  -0.18  2.34
1   Blue collar              Wealth  238  -0.62  5.61  275  -0.31  5.84
2   Blue collar                 Age  238   1.08  0.81  275   1.07  0.80
3   Blue collar  Years of Schooling  238   0.18  2.91  275  -0.25  3.14
4  White collar                Wage  225  -0.33  2.39  259   0.11  2.18
5  White collar              Wealth  225  -0.15  5.48  259  -0.20  5.41
6  White collar                 Age  225   1.04  0.81  259   0.98  0.81
7  White collar  Years of Schooling  225  -0.45  2.99  259  -0.00  3.10, _body=<great_tables._gt_data.Body object at 0x000001BC66FA57D0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='N', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='Std. Dev.', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='N', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='Std. Dev.', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC68514DD0>, _spanners=Spanners([SpannerInfo(spanner_id='EU', spanner_level=1, spanner_label='EU', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='US', spanner_level=1, spanner_label='US', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title='Descriptive statistics', subtitle=None, preheader=None), _stubhead=None, _source_notes=[None], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC6869AB50>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_b

# Table Layout for DataFrames with `pf.make_table()`

`pf.make_table()` is called by `pf.etable()` and `pf.dtable()` to generate the tables in "gt" and "tex" format. But you can also use it directly to generate tables in the same layout from other pandas dataframes. 

## Basic Usage of make_table

In [26]:
df = pd.DataFrame(np.random.randn(4, 4).round(2), columns=["A", "B", "C", "D"])

# Make Booktabs style table
pf.make_table(df=df, caption="This is a caption", notes="These are notes")

GT(_tbl_data=   index     A     B     C     D
0      0 -0.60 -0.55 -0.82 -0.35
1      1 -0.91  0.67 -0.50  2.74
2      2  0.25  1.13 -0.92 -0.51
3      3  0.76  1.12 -0.27 -0.72, _body=<great_tables._gt_data.Body object at 0x000001BC68581490>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='center', column_width=None), ColInfo(var='A', type=<ColInfoTypeEnum.default: 1>, column_label='A', column_align='center', column_width=None), ColInfo(var='B', type=<ColInfoTypeEnum.default: 1>, column_label='B', column_align='center', column_width=None), ColInfo(var='C', type=<ColInfoTypeEnum.default: 1>, column_label='C', column_align='center', column_width=None), ColInfo(var='D', type=<ColInfoTypeEnum.default: 1>, column_label='D', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC6865CB10>, _spanners=Spanners([]), _heading=Heading(title='This is a caption', subtitle=None, preheader=None), _stubhead=None, _source_notes=['These are notes'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC685923D0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), heading_t

## Mutiindex DataFrames
When the respective dataframe has a mutiindex for the columns, columns spanners are generated from the index. The row index can also be a multiindex (of at most two levels). In this case the first index level is used to generate group rows (for instance using the index name as headline and separating the groups by a horizontal line) and the second index level is used to generate the row labels.

In [27]:
# Create a multiindex dataframe with random data
row_index = pd.MultiIndex.from_tuples(
    [
        ("Group 1", "Variable 1"),
        ("Group 1", "Variable 2"),
        ("Group 1", "Variable 3"),
        ("Group 2", "Variable 4"),
        ("Group 2", "Variable 5"),
        ("Group 3", "Variable 6"),
    ]
)

col_index = pd.MultiIndex.from_product([["A", "B"], ["X", "Y"], ["High", "Low"]])
df = pd.DataFrame(np.random.randn(6, 8).round(3), index=row_index, columns=col_index)

pf.make_table(df=df, caption="This is a caption", notes="These are notes")

GT(_tbl_data=   level_0     level_1      0      1      2      3      4      5      6      7
0  Group 1  Variable 1 -0.296  0.786 -0.803  0.232  0.654  0.519 -0.772  0.491
1  Group 1  Variable 2  0.305  0.471 -0.107 -1.184  0.537  1.412  0.364  1.011
2  Group 1  Variable 3 -1.468 -0.733  0.195 -0.196  0.596  0.356 -0.877  1.589
3  Group 2  Variable 4  1.394 -0.558 -2.120  1.259 -0.109  0.392 -1.473  0.016
4  Group 2  Variable 5 -0.027 -1.263  1.446 -0.937 -0.180 -0.131 -0.724 -1.652
5  Group 3  Variable 6  1.522  0.690 -0.798  0.342  0.649  1.015  1.337 -0.388, _body=<great_tables._gt_data.Body object at 0x000001BC6862C5D0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='High', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='Low', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='High', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='Low', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='High', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='Low', column_align='center', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='High', column_align='center', column_width=None), ColInfo(var='7', type=<ColInfoTypeEnum.default: 1>, column_label='Low', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC6858CD90>, _spanners=Spanners([SpannerInfo(spanner_id='A', spanner_level=2, spanner_label='A', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2', '3'], built=None), SpannerInfo(spanner_id='B', spanner_level=2, spanner_label='B', spanner_units=None, spanner_pattern=None, vars=['4', '5', '6', '7'], built=None), SpannerInfo(spanner_id='X', spanner_level=1, spanner_label='X', spanner_units=None, spanner_pattern=None, vars=['0', '1', '4', '5'], built=None), SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['2', '3', '6', '7'], built=None)]), _heading=Heading(title='This is a caption', subtitle=None, preheader=None), _stubhead=None, _source_notes=['These are notes'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC6858EF50>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True


You can also hide column group names: This just creates a table where variables on the second level of the row index are displayed in groups based on the first level separated by horizontal lines.

In [28]:
pf.make_table(
    df=df, caption="This is a caption", notes="These are notes", rgroup_display=False
).tab_style(style=style.text(style="italic"), locations=loc.body(rows=[1, 5]))

GT(_tbl_data=   level_0     level_1      0      1      2      3      4      5      6      7
0  Group 1  Variable 1 -0.296  0.786 -0.803  0.232  0.654  0.519 -0.772  0.491
1  Group 1  Variable 2  0.305  0.471 -0.107 -1.184  0.537  1.412  0.364  1.011
2  Group 1  Variable 3 -1.468 -0.733  0.195 -0.196  0.596  0.356 -0.877  1.589
3  Group 2  Variable 4  1.394 -0.558 -2.120  1.259 -0.109  0.392 -1.473  0.016
4  Group 2  Variable 5 -0.027 -1.263  1.446 -0.937 -0.180 -0.131 -0.724 -1.652
5  Group 3  Variable 6  1.522  0.690 -0.798  0.342  0.649  1.015  1.337 -0.388, _body=<great_tables._gt_data.Body object at 0x000001BC6862D6D0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='High', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='Low', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='High', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='Low', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='High', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='Low', column_align='center', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='High', column_align='center', column_width=None), ColInfo(var='7', type=<ColInfoTypeEnum.default: 1>, column_label='Low', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC66D78AD0>, _spanners=Spanners([SpannerInfo(spanner_id='A', spanner_level=2, spanner_label='A', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2', '3'], built=None), SpannerInfo(spanner_id='B', spanner_level=2, spanner_label='B', spanner_units=None, spanner_pattern=None, vars=['4', '5', '6', '7'], built=None), SpannerInfo(spanner_id='X', spanner_level=1, spanner_label='X', spanner_units=None, spanner_pattern=None, vars=['0', '1', '4', '5'], built=None), SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['2', '3', '6', '7'], built=None)]), _heading=Heading(title='This is a caption', subtitle=None, preheader=None), _stubhead=None, _source_notes=['These are notes'], _footnotes=[], _styles=[StyleInfo(locname='data', locnum=5, grpname=None, colname='0', rownum=1, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='0', rownum=5, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='1', rownum=1, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='1', rownum=5, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname=None, colname='2', rownum=1, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname='data', locnum=5, grpname

# Custom Styling with Great Tables
You can use the rich set of methods offered by [Great Tables](https://posit-dev.github.io/great-tables/articles/intro.html) to further customize the table display when the type is "gt".

## Example Styling


In [29]:
(
    pf.etable([fit1, fit2, fit3, fit4, fit5, fit6])
    .tab_options(
        column_labels_background_color="cornsilk",
        stub_background_color="whitesmoke",
    )
    .tab_style(
        style=style.fill(color="mistyrose"),
        locations=loc.body(columns="(3)", rows=["X2"]),
    )
)

GT(_tbl_data=  level_0        level_1                       0                       1                       2                       3                       4                       5
0    coef             X1  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)  -1.231*** <br> (0.192)
1    coef             X2  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)     -0.074 <br> (0.104)
2    coef          X1:X2                                                      0.011 <br> (0.018)                                                     -0.041 <br> (0.081)
3      fe             f1                       x                       x                       x                       x                       x                       x
4      fe             f2                       -                       x                       x                       -                       x                       x
5   stats   Observations                     997                     997                     997                     998                     998                     998
6   stats      S.E. type                  by: f1                  by: f1                  by: f1                  by: f1                  by: f1                  by: f1
7   stats  R<sup>2</sup>                   0.489                   0.659                   0.659                   0.120                   0.172                   0.172, _body=<great_tables._gt_data.Body object at 0x000001BC66D2C690>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC66C66510>, _spanners=Spanners([SpannerInfo(spanner_id='Y', spanner_level=1, spanner_label='Y', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Y2', spanner_level=1, spanner_label='Y2', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC68537E10>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 

## Defining Table Styles

You can easily define table styles that you can apply to all tables in your project. Just define a dictionary with the respective values for the tab options (see the [Great Tables documentation](https://posit-dev.github.io/great-tables/reference/GT.tab_options.html#great_tables.GT.tab_options)) and use the style with `.tab_options(**style_dict)`.

In [30]:
style_print = {
    "table_font_size": "12px",
    "heading_title_font_size": "12px",
    "source_notes_font_size": "8px",
    "data_row_padding": "3px",
    "column_labels_padding": "3px",
}

style_presentation = {
    "table_font_size": "16px",
    "table_font_color_light": "white",
    "table_body_border_top_style": "hidden",
    "table_body_border_bottom_style": "hidden",
    "heading_title_font_size": "18px",
    "source_notes_font_size": "12px",
    "data_row_padding": "3px",
    "column_labels_padding": "6px",
    "column_labels_background_color": "midnightblue",
    "stub_background_color": "whitesmoke",
    "row_group_background_color": "whitesmoke",
    "table_background_color": "whitesmoke",
    "heading_background_color": "white",
    "source_notes_background_color": "white",
    "column_labels_border_bottom_color": "white",
    "column_labels_font_weight": "bold",
    "row_group_font_weight": "bold",
}

In [31]:
t1 = pf.dtable(
    data,
    vars=["Y", "Y2", "X1", "X2"],
    stats=["count", "mean", "std", "min", "max"],
    labels=labels,
    caption="Descriptive statistics",
)

t2 = pf.etable(
    [fit1, fit2, fit3, fit4, fit5, fit6],
    labels=labels,
    show_se=False,
    felabels={"f1": "Industry Fixed Effects", "f2": "Year Fixed Effects"},
    caption="Regression results",
)

In [32]:
display(t1.tab_options(**style_print))
display(t2.tab_options(**style_print))

GT(_tbl_data=                index    N   Mean Std. Dev.     Min    Max
0                Wage  997  -0.13      2.31   -6.54   6.91
1              Wealth  997  -0.32      5.59  -16.97  17.16
2                 Age  997   1.04      0.81    0.00   2.00
3  Years of Schooling  997  -0.13      3.05   -9.67  10.99, _body=<great_tables._gt_data.Body object at 0x000001BC68514490>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='center', column_width=None), ColInfo(var='N', type=<ColInfoTypeEnum.default: 1>, column_label='N', column_align='center', column_width=None), ColInfo(var='Mean', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='center', column_width=None), ColInfo(var='Std. Dev.', type=<ColInfoTypeEnum.default: 1>, column_label='Std. Dev.', column_align='center', column_width=None), ColInfo(var='Min', type=<ColInfoTypeEnum.default: 1>, column_label='Min', column_align='center', column_width=None), ColInfo(var='Max', type=<ColInfoTypeEnum.default: 1>, column_label='Max', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC66D2DA10>, _spanners=Spanners([]), _heading=Heading(title='Descriptive statistics', subtitle=None, preheader=None), _stubhead=None, _source_notes=[None], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC66D2CA50>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='12px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=

GT(_tbl_data=  level_0                        level_1                       0                       1                       2                       3                       4                       5
0    coef                            Age  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)  -1.231*** <br> (0.192)
1    coef             Years of Schooling  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)     -0.074 <br> (0.104)
2    coef  Age &#215; Years of Schooling                                                      0.011 <br> (0.018)                                                     -0.041 <br> (0.081)
3      fe         Industry Fixed Effects                       x                       x                       x                       x                       x                       x
4      fe             Year Fixed Effects                       -                       x                       x                       -                       x                       x
5   stats                   Observations                     997                     997                     997                     998                     998                     998
6   stats                      S.E. type                  by: f1                  by: f1                  by: f1                  by: f1                  by: f1                  by: f1
7   stats                  R<sup>2</sup>                   0.489                   0.659                   0.659                   0.120                   0.172                   0.172, _body=<great_tables._gt_data.Body object at 0x000001BC66EC9390>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC6844E010>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title='Regression results', subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC6844F410>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=Tr

In [33]:
display(t1.tab_options(**style_presentation))
display(t2.tab_options(**style_presentation))

GT(_tbl_data=                index    N   Mean Std. Dev.     Min    Max
0                Wage  997  -0.13      2.31   -6.54   6.91
1              Wealth  997  -0.32      5.59  -16.97  17.16
2                 Age  997   1.04      0.81    0.00   2.00
3  Years of Schooling  997  -0.13      3.05   -9.67  10.99, _body=<great_tables._gt_data.Body object at 0x000001BC68514490>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='center', column_width=None), ColInfo(var='N', type=<ColInfoTypeEnum.default: 1>, column_label='N', column_align='center', column_width=None), ColInfo(var='Mean', type=<ColInfoTypeEnum.default: 1>, column_label='Mean', column_align='center', column_width=None), ColInfo(var='Std. Dev.', type=<ColInfoTypeEnum.default: 1>, column_label='Std. Dev.', column_align='center', column_width=None), ColInfo(var='Min', type=<ColInfoTypeEnum.default: 1>, column_label='Min', column_align='center', column_width=None), ColInfo(var='Max', type=<ColInfoTypeEnum.default: 1>, column_label='Max', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC66D2DA10>, _spanners=Spanners([]), _heading=Heading(title='Descriptive statistics', subtitle=None, preheader=None), _stubhead=None, _source_notes=[None], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC66D2CA50>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='whitesmoke'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='white'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color

GT(_tbl_data=  level_0                        level_1                       0                       1                       2                       3                       4                       5
0    coef                            Age  -0.950*** <br> (0.067)  -0.924*** <br> (0.061)  -0.924*** <br> (0.061)  -1.267*** <br> (0.174)  -1.232*** <br> (0.192)  -1.231*** <br> (0.192)
1    coef             Years of Schooling  -0.174*** <br> (0.018)  -0.174*** <br> (0.015)  -0.185*** <br> (0.025)   -0.131** <br> (0.042)   -0.118** <br> (0.042)     -0.074 <br> (0.104)
2    coef  Age &#215; Years of Schooling                                                      0.011 <br> (0.018)                                                     -0.041 <br> (0.081)
3      fe         Industry Fixed Effects                       x                       x                       x                       x                       x                       x
4      fe             Year Fixed Effects                       -                       x                       x                       -                       x                       x
5   stats                   Observations                     997                     997                     997                     998                     998                     998
6   stats                      S.E. type                  by: f1                  by: f1                  by: f1                  by: f1                  by: f1                  by: f1
7   stats                  R<sup>2</sup>                   0.489                   0.659                   0.659                   0.120                   0.172                   0.172, _body=<great_tables._gt_data.Body object at 0x000001BC66EC9390>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001BC6844E010>, _spanners=Spanners([SpannerInfo(spanner_id='Wage', spanner_level=1, spanner_label='Wage', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2'], built=None), SpannerInfo(spanner_id='Wealth', spanner_level=1, spanner_label='Wealth', spanner_units=None, spanner_pattern=None, vars=['3', '4', '5'], built=None)]), _heading=Heading(title='Regression results', subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001BC6844F410>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=Tr